In [2]:
import ants
import nibabel as nib
import numpy as np

In [24]:
avgt = nib.load("/Users/mahdi/stage-2022-mahdi/data/AVGT.nii.gz")
avgt_data = avgt.get_fdata()
avgt_affine = avgt.affine
avgt_shape = avgt.shape

In [3]:
fname = "/Users/mahdi/Desktop/tif10.nii.gz"

In [4]:
ofname = "/Users/mahdi/Desktop/tif10_oriented.nii.gz"

In [5]:
ls10 = nib.load(fname)

In [6]:
ls10_array = ls10.get_fdata()
ls10_affine = ls10.affine
ls10_shape = ls10.shape

## PSR to RAS

In [7]:
def switch_axis(vol):
    vol = np.moveaxis(vol, (0, 1, 2), (2, 0, 1))
    vol = np.flip(vol, axis=2)
    return vol

In [8]:
ls10_array = switch_axis(ls10_array)

In [9]:
ls10_oriented = nib.Nifti1Image(ls10_array, ls10_affine)

In [10]:
nib.save(ls10_oriented,ofname)

In [25]:
ls_oriented_avgt = nib.Nifti1Image(ls10_array, avgt_affine)

In [26]:
nib.save(ls_oriented_avgt,"/Users/mahdi/Desktop/tif10_oriented_affine.nii.gz")

## registration

In [31]:
fvol = avgt_data.astype(np.float32)
mvol = ls10_array.astype(np.float32)

In [32]:
fi = ants.from_numpy(fvol).resample_image(avgt_shape, 1, 0)
mi = ants.from_numpy(mvol).resample_image(avgt_shape, 1, 0)

In [15]:
movmask = ants.get_mask(mi)

In [59]:
mytx = ants.registration(
    fixed=fi,
    moving=mi,
    type_of_transform='SyN'
)

In [60]:
wmi = ants.apply_transforms(
    fixed=fi,
    moving=mi,
    transformlist=mytx['fwdtransforms'],
    interpolator='bSpline'
)

In [61]:
ls10_aligned = nib.Nifti1Image(wmi.numpy(),avgt_affine)

In [62]:
aname = "/Users/mahdi/Desktop/tif10_aligned.nii.gz"

In [63]:
nib.save(ls10_aligned,aname)

## analyse du lightsheet ras